# Classical Phase Estimation (with QFT) Module

The present notebook reviews the classical **Quantum Phase Estimation Algorithm**, which utilizes the inverse of the **Quantum Fourier Transform** ($\mathcal{QFT}^\dagger$) to estimate the eigenvalues (phases) of a unitary operator. From now on, we will refer to this algorithm as **CQPE**. The **CQPE** algorithm is implemented in the module `classical_qpe.py` of the package `PE` within the *QQuantLib* library (**QQuantLib/PE/classical_qpe.py**).

This algorithm was developed as a Python class named `CQPE` inside the file **QQuantLib/PE/classical_qpe.py**.

---

## References

The content of this notebook and its associated module are based on the following references:

- **Brassard, G., Hoyer, P., Mosca, M., & Tapp, A. (2000).** Quantum amplitude amplification and estimation. AMS Contemporary Mathematics Series, 305. [arXiv:quant-ph/0005055v1](https://arxiv.org/abs/quant-ph/0005055v1)
  
- **NEASQC Deliverable:** D5.1: Review of state-of-the-art for Pricing and Computation of VaR. [PDF](https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf)

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus: python, c
# QLM qpus accessed using Qaptiva Access library: qlmass_linalg, qlmass_mps
# QLM qpus: Only in local Quantum Learning Machine: linalg, mps
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]

linalg_qpu = get_qpu(my_qpus[1])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Example to use

To illustrate how the `CQPE` class works, we will use the **Iterative Quantum Phase Estimation (IQPE)** example from the Qiskit textbook:

- [Qiskit Tutorial: IQPE](https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb)

This notebook contains 2 different examples:
- **IPE example with a 1-qubit gate for U**
- **IPE example with a 2-qubit gate**

The sections 2 and 3 will explain, step by step, how to use the `CQPE` class for solve the **IPE example with a 1-qubit gate for U** example. 

Section 4 will explain how to use the library solve the **IPE example with a 2-qubit gate**.

## 2. State and Unitary operator.

In the **IPE example with a 1-qubit gate for U** from the Qiskit textbook, the main problem is to find the phase for the 1-qubit unitary operator. They use the  $\mathcal{S}$ gate that has the following behaviour:

$$
\mathcal{S}|1\rangle = e^{i\frac{\pi}{2}}|1\rangle.
$$

This means that the $\mathcal{S}$ gate introduces a phase shift of $\frac{\pi}{2}$ radians to the state $|1\rangle$, leaving it unchanged otherwise.

To use the `CQPE` Python class from the module **QQuantLib/PE/classical_pe**, two mandatory inputs must be provided:

1. **Initial State**: This is the initial quantum state required for applying the unitary operator ($|1\rangle$ in the Qiskit example). The initial state can be provided as:
    - A QLM QRoutine.
    - A QLM gate (or abstract gate).
2. **Unitary Operator**: This is the operator whose phase we aim to estimate ($\mathcal{S}$ gate in the Qiskit example). The unitary operator can be provided as:
    - A QLM QRoutine.
    - A QLM gate (or abstract gate).    



In [ ]:
# Initial quantum state
#Number Of Qubits
n_qbits = 1
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state --svg

In [ ]:
# unitary operator
unitary_operator = qlm.PH(np.pi/2.0)  

In [ ]:
%qatdisplay unitary_operator --svg

## 3. Class CQPE: Classical Quantum Phase Estimation Algorithm Step by Step

The problem of phase estimation can be formulated as follows. Given an initial state $|\Psi\rangle$ and a phase operator $\mathcal{Q}$ such that:

$$
\mathcal{Q}|\Psi\rangle = e^{2\pi i \lambda}|\Psi\rangle,
$$

our objective is to estimate the value of $\lambda$.

So far, we have the initial state $|\Psi\rangle = |1\rangle$ and the unitary operator whose phase we aim to estimate, $\mathcal{Q} = \mathcal{S}$. In this section, we will describe the class step by step and explain the fundamentals of the **CQPE** algorithm.

---

### 3.1 Calling the `CQPE` Class

The `CQPE` class is located in the **QQuantLib/PE/classical_qpe** module.

To instantiate the class, the user must provide a Python dictionary with the following mandatory keys:

- `initial_state`: A QLM routine or gate with an initial state $|\Psi\rangle$ (created in Section 1).
- `unitary_operator`: A QLM gate or routine representing the unitary operator to be applied to the initial state $|\Psi\rangle$ (also created in Section 1).

Additionally, the following optional keys can be used to configure the method:

- `auxiliar_qbits_number`: The number of auxiliary qubits ($m$) used for phase estimation (default: 8). This determines the precision of the phase estimation, which scales as $\frac{1}{2^m}$.
- `qpu`: myQLM or QLM QPU. If not provided, the class attempts to create a PyLinalg solver. It is highly recommended to specify this key.
- `shots`: The number of measurement shots for the quantum job (default: 10).
- `complete`: If set to `True`, all possible states will be returned when `shots` is greater than zero.

These parameters allow users to customize the behavior of the algorithm according to their specific requirements.

In [ ]:
#Load Class
from QQuantLib.PE.classical_qpe import CQPE

In [ ]:
auxiliar_qbits_number = 2
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : auxiliar_qbits_number,
    #'complete': True,
    'shots':10
}
qft_pe = CQPE(**qft_pe_dict)

When the class is instantiated the properties *initial_state* and *q_gate* are overwritten with the given keys **initial_state** and **unitary_operator** respectively

In [ ]:
c = qft_pe.initial_state
%qatdisplay c --svg

In [ ]:
a = qft_pe.q_gate
%qatdisplay a --depth 2 --svg

### 3.2 Classical Quantum Phase Estimation Algorithm

The classical **QPE** algorithm consists of the following steps:

1. **Allocate State Qubits**: Determine the number of qubits required for the initial state operator $\mathcal{Q}$.
2. **Allocate Auxiliary Qubits**: Allocate $m$ additional qubits, where the phase information will be encoded.
3. **Apply Hadamard Gates**: Apply a Hadamard gate to each auxiliary qubit to create a superposition of states.
4. **Controlled Unitary Operations**: Each auxiliary qubit applies a controlled $2^i$ power of the unitary operator to the initial state. Here, $i$ corresponds to the position of the auxiliary qubit.
5. **Inverse Quantum Fourier Transform**: Apply the inverse of the Quantum Fourier Transform ($\mathcal{QFT}^\dagger$) to the auxiliary qubits.
6. **Measurement**: Measure the auxiliary qubits and convert the measurement outcomes into an integer value $y$.
7. **Compute Phase**: Calculate the estimated phase using the formula:
   $$
   \lambda = \frac{y}{2^m}.
   $$

To execute this workflow, the **run** method of the class should be invoked. The resulting measurements are stored in a pandas DataFrame within the class attribute `result`, which includes the following columns:

- `States`: Possible quantum states of the auxiliary qubits.
- `Int_lsb`: Conversion of the quantum state to an integer following the **least significant bit (LSB)** convention (the rightmost bit is the least significant).
- `Probability`: Computed frequency of each quantum state.
- `Int`: Conversion of the quantum state to an integer following the **most significant bit (MSB)** convention (the rightmost bit is the most significant).
- `lambda`: The estimated phase obtained from the measurements.

The contents of the `result` DataFrame depend on the values of the input variables `shots` and `complete`:
- If `shots` is set to zero or `complete` is `True`, the DataFrame will include all possible quantum states.
- If `shots` is greater than zero and `complete` is `False`, only the measured states will be included in the results.

In [ ]:
qft_pe.run()

In [ ]:
c = qft_pe.circuit
%qatdisplay c --svg

In [ ]:
qft_pe.result

In the case of the *Qiskit* example provided:

$$
i\frac{\pi}{2} = 2\pi i \lambda,
$$

the desired solution corresponds to:

$$
\lambda = \frac{1}{4}.
$$

In the previous scenario, we set the *complete* variable to **True**, ensuring that all possible states are returned. In this context, the $\lambda$ value is determined by selecting the state with the highest probability, yielding:

$$
\lambda = 0.25.
$$

This result aligns with the expected outcome from the *Qiskit* example, confirming the accuracy of our phase estimation process.

In [ ]:
qft_pe.result.sort_values('Probability', ascending=False).iloc[0]['lambda']

If `shots` is different of zero and  `complete` is set to `False` only the measured states will be returned!!

In [ ]:
auxiliar_qbits_number = 2
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : auxiliar_qbits_number,
    'complete': False,
    'shots':100
}
qft_pe = CQPE(**qft_pe_dict)
qft_pe.run()
qft_pe.result

Addtitionally the attribute `quantum_times` measures the time for the quantum part of the algorithm (or simulated)

In [ ]:
qft_pe.quantum_times

## 4.  IPE example with a 2-qubit gate

We will now reproduce the Qiskit example from the section **IPE example with a 2-qubit gate** (available in [Qiskit Tutorials](https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb)).

In this scenario, the example uses **2 qubits** to estimate the phase of a unitary operator $\mathcal{cT}$. This operator introduces a phase shift of $\frac{\pi}{4}$ radians specifically to the state $|11\rangle$, leaving all other states unchanged. Mathematically, this behavior is expressed as:

$$
\mathcal{cT} |11\rangle = e^{i\frac{\pi}{4}} |11\rangle.
$$

From the phase estimation convention, we can derive the relationship between the phase shift and the parameter $\lambda$:

$$
i\frac{\pi}{4} = 2\pi i \lambda.
$$

Solving for $\lambda$, we obtain:

$$
\lambda = \frac{1}{8}.
$$

This value of $\lambda$ represents the desired solution for the phase estimation problem in this example.

In [ ]:
#create initial state
n_qbits = 2
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state --svg

In [ ]:
#Create cT operator
unitary_operator = qlm.QRoutine()
uq_qbits = unitary_operator.new_wires(n_qbits)
unitary_operator.apply(qlm.PH(np.pi/4.0).ctrl(), 0, 1)

In [ ]:
%qatdisplay unitary_operator --svg

In [ ]:
#now PE with QFT!!
n_cbits = 3
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : n_cbits,  
    'shots': 100,
    'complete': False
}
qft_pe = CQPE(**qft_pe_dict)

In [ ]:
qft_pe.run()

In [ ]:
qft_pe.result.sort_values('Probability', ascending = False)

The result for $\varphi=0.125$ that is the value obtained in the Qiskit example.

## 5. Application to Amplitude Estimation

The problem of **Amplitude Estimation** can be formulated as follows: Given an oracle operator $\mathcal{A}$ with the following behavior:

$$
|\Psi\rangle = \mathcal{A}|0\rangle = \sqrt{a}|\Psi_0\rangle + \sqrt{1-a}|\Psi_1\rangle,
$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, our goal is to estimate $\sqrt{a}$. 

A naive solution involves measuring the system $N$ times to estimate the probability of obtaining $|\Psi_0\rangle$. In this case, the error in estimating $\sqrt{a}$ scales as:

$$
\epsilon_a \sim \frac{1}{\sqrt{N}}.
$$

It is often useful to express this error in terms of the number of oracle calls, $N_{\mathcal{A}}$. For the naive approach, it can be shown that:

$$
\epsilon_a \sim \frac{1}{\sqrt{N_{\mathcal{A}}}}.
$$

However, the classical **Phase Estimation** algorithm with **QFT** can achieve a quadratic speedup in solving **Amplitude Estimation** problems.

To apply the **Phase Estimation** algorithm to **Amplitude Estimation**, the following steps should be followed:

1. **Substitution**: Replace $\sqrt{a}$ with $\sin(\theta)$:
   $$
   \sqrt{a} = \sin(\theta).
   $$

2. **Rewriting the Oracle Operator**: The action of the oracle operator becomes:
   $$
   |\Psi\rangle = \mathcal{A}|0\rangle = \sqrt{a}|\Psi_0\rangle + \sqrt{1-a}|\Psi_1\rangle = \sin(\theta)|\Psi_0\rangle + \cos(\theta)|\Psi_1\rangle. \tag{1}
   $$

3. **Creating the Grover-like Operator**: Construct the Grover-like operator for the oracle operator $\mathcal{A}$:
   $$
   \mathbf{G}(\mathcal{A}) = \mathcal{A} \left(\hat{I} - 2|0\rangle\langle 0|\right) \mathcal{A}^\dagger \left(\hat{I} - 2|\Psi_0\rangle\langle \Psi_0|\right).
   $$

With these substitutions, the classical **Phase Estimation** algorithm can now be applied directly by replacing:

$$
\begin{aligned}
& |\Psi\rangle \longrightarrow \mathcal{A}|0\rangle, \\
& \mathcal{Q} \longrightarrow \mathbf{G}(\mathcal{A}).
\end{aligned}
$$

It can be demonstrated that the $\mathbf{G}(\mathcal{A})$ operator represents a rotation in the plane by an angle $2\theta$, expressed as:

$$
\mathbf{G}(\mathcal{A}) =
\begin{pmatrix}
\cos(2\theta) & -\sin(2\theta) \\
\sin(2\theta) & \cos(2\theta)
\end{pmatrix}.
$$

Thus, the eigenvalues of $\mathbf{G}(\mathcal{A})$ are given by $e^{\pm i2\theta}$.

Using the phase estimation convention:

$$
i2\theta = 2\pi i \lambda,
$$

we arrive at:

$$
\theta = \pi \lambda.
$$

It can be shown that when using the classical **Phase Estimation** algorithm, the error in estimating $\theta$ (and consequently $\sqrt{a}$) scales as:

$$
\epsilon_\theta \sim \frac{1}{N_{\mathcal{A}}}.
$$

This quadratic improvement in error scaling demonstrates the significant advantage of the **Phase Estimation** algorithm over the naive solution.

### 5.1 Using *CQPE* class for *Amplitude Estimation*

In this section, we will demonstrate how to use the **CQPE** class to solve **Amplitude Estimation** problems.

#### Defining the Amplitude Estimation Problem

We begin by defining the following amplitude estimation problem:

$$
|\Psi\rangle = \mathcal{A}|0\rangle = \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[\sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right]. \tag{2}
$$

By comparing equation (2) with the general form of the state in equation (1):

$$
\sqrt{a}|\Psi_0\rangle = \sin(\theta)|\Psi_0\rangle = \frac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle,
$$

and

$$
\sqrt{1-a}|\Psi_1\rangle = \cos(\theta)|\Psi_1\rangle = \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[\sqrt{0}|0\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right].
$$

Here:
- The target state $|\Psi_0\rangle$ corresponds to $|1\rangle$, and its amplitude is proportional to $\sin(\theta)$.
- The remaining states collectively form $|\Psi_1\rangle$, with an amplitude proportional to $\cos(\theta)$.

#### Creating the Initial State

The subsequent cells will construct the initial state $|\Psi\rangle = \mathcal{A}|0\rangle$, which serves as the starting point for our amplitude estimation process. This involves loading the specified probability distribution into a quantum state using the appropriate functions from the library.

In [ ]:
from QQuantLib.DL.data_loading import load_probability
from QQuantLib.AA.amplitude_amplification import grover

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

Now we need to compute the Grover-like operator from the oracle operator $\mathcal{A}$. The functions from `AA` module will be used (see notebook **02_Amplitude_Amplification_Operators.ipynb** for more information)

In [ ]:
target = [0, 0, 1]
index = range(oracle.arity)
grover_gate = grover(oracle, target, index)
#Comment before line and uncomment following for For multiplexor implementation of multi-controlled Z gate
#grover_gate = grover(oracle, target, index, mcz_qlm=False)
%qatdisplay grover_gate --depth 2 --svg

Now that we have translated our amplitude amplification problem to an phase estimation problem we proceed to use our class normally. We provide the *oracle* as the **initial_state** and the correspondent Grover-like operator as the **unitary_operator**. Additionally `auxiliar_qbits_number` for estimating the phase should be provided.

In [ ]:
n_cbits = 8
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': oracle,
    'unitary_operator': grover_gate,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : n_cbits,  
    'shots': 1000,
    'complete':True #For getting all the posible 2^n_cbits states
}
qft_pe = CQPE(**qft_pe_dict)
qft_pe.run()

In [ ]:
#Using the circuit attribute for draw the complete routine
c = qft_pe.circuit
%qatdisplay c --svg

In [ ]:
#We can plot the obtained the frequency vs the theta
qft_pe.result.sort_values('lambda', inplace=True)
plt.plot(qft_pe.result['lambda'], qft_pe.result['Probability'], 'o-')
plt.xlabel(r'$\lambda$')
plt.ylabel('Probability')

Now we can compute $\theta = \pi \lambda$.

In [ ]:
qft_pe.result['theta'] = np.pi * qft_pe.result['lambda']

In [ ]:
#We can plot the obtained the frequency vs the theta in the [0, pì/2] range
qft_pe.result.sort_values('theta', inplace=True)
plt.plot(qft_pe.result['theta'], qft_pe.result['Probability'], 'o-')
plt.xlabel(r'$\theta$')
plt.ylabel('Probability')

In this case, the 2 obtained maximums reflect the 2 *eigenvalues* of $\mathbf{G}(\mathcal{A})$: $\pm\theta$. In general it is useful to restrict $\theta$ to the interval $(0, \frac{\pi}{2}$). This can be done in the following cell:

In [ ]:
qft_pe.result['theta_90'] = qft_pe.result['theta']
qft_pe.result['theta_90'].where(
    qft_pe.result["theta_90"] < 0.5 * np.pi,
    np.pi - qft_pe.result["theta_90"],
    inplace=True,
)

In [ ]:
#We can plot the obtained the frequency vs the theta in the [0, pì/2] range
qft_pe.result.sort_values('theta_90', inplace=True)
plt.plot(qft_pe.result['theta_90'], qft_pe.result['Probability'], 'o-')
plt.xlabel(r'$\theta$')
plt.ylabel('Probability')
plt.title(r'$\theta \in [0, \frac{\pi}{2}]$')

Finally, we need to compute the desired value of $a$. Based on the equations presented in the notebook, this can typically be calculated as:

$$
a = \sin(\theta)^2.
$$

**However, in our implementation of the Quantum Phase Estimation (QPE) algorithm, the convention has been modified. The following relationship MUST BE USED instead:**

$$
a = \cos(\theta)^2.
$$

This adjustment ensures consistency with the internal logic and conventions adopted in the implementation of the **CQPE** class. By using this modified relationship, we align the estimated amplitude $a$ with the expected results from the algorithm.

In [ ]:
qft_pe.result['a']=np.cos(qft_pe.result['theta_90'])**2

In [ ]:
#We can plot the obtained a vs the theta in the [0, pì/2] range
plt.plot(qft_pe.result['a'], qft_pe.result['Probability'], 'o-')
plt.xlabel(r'$a$')
plt.ylabel('Probability')
plt.title(r'$a \in [0, 1]$')

### What *a* Should Be Returned?

From the perspective of the **QPE** algorithm applied to **AE**, one straightforward solution would be to return the most frequently measured value of $a$. However, in practice, this approach can be problematic due to imperfections in actual quantum computers. Current quantum hardware is prone to noise and errors, which may prevent the **QPE** algorithm from executing perfectly. As a result, the measured outcomes might deviate from the expected results of the ideal algorithm.

To address this challenge, we propose using the **mean** of the probability distribution obtained from the measurements. This approach provides a more robust estimate of $a$, even in the presence of experimental uncertainties. The mean value $\tilde{a}$ is computed as follows:

$$
\tilde{a} = \int a \cdot p(a) \, da = \sum_{i} a[i] \cdot P(a[i]),
$$

where:
- $p(a)$ represents the probability of obtaining the value $a$.
- Experimentally, $P(a[i])$ corresponds to the frequency with which each value $a[i]$ was observed, normalized by the total number of measurement shots executed.

By calculating the mean of the measured distribution, we obtain a more reliable estimate of the amplitude $a$, accounting for potential inaccuracies introduced by the quantum hardware.

In [ ]:
qft_pe.result.sort_values('Probability', inplace=True, ascending=False)
mean_a = qft_pe.result['Probability'] @ qft_pe.result['a']

In [ ]:
print("Most frequent value of a: {}".format(qft_pe.result['a'].iloc[0]))
print("Mean value of a: {}".format(mean_a))
print("Classical result: ",probability[1])
print('Test OK. Most Frequent Value: ', probability[1]- qft_pe.result['a'].iloc[0] < 0.005)
print('Test OK. Mean Value: ', probability[1] - qft_pe.result['Probability'] @ qft_pe.result['a'] < 0.005)

### Associated Error of $a$

In addition to estimating the value of $a$, we can compute an error associated with the estimator $\tilde{a}$, denoted as $\epsilon_a$. This error accounts for uncertainties in the estimation process and helps assess the reliability of the result.

#### Ideal Quantum Computer:
For the **Quantum Phase Estimation (QPE)** algorithm executed on an ideal quantum computer, the error depends on the number of auxiliary qubits ($m$) used for phase estimation. Specifically, the error is bounded by:

$$
\epsilon_a^{QPE} < \frac{2\pi}{2^m}.
$$

This bound reflects the precision achievable with $m$ auxiliary qubits, where the phase estimation resolution improves exponentially with $m$.

#### Actual Quantum Computer:
On real quantum hardware, errors are more pronounced due to noise and imperfections. In such cases, the most straightforward approach to estimate the experimental error is to calculate the square root of the variance of the measured probability density. This experimental error, $\epsilon_a^{P(a)}$, is given by:

$$
\epsilon_a^{P(a)} = \sqrt{\int p(a) (a - \tilde{a})^2 \, da} = \sqrt{\sum_{i} (a[i] - \tilde{a})^2 \cdot P(a[i])},
$$

where:
- $p(a)$ represents the probability density function of $a$,
- $\tilde{a}$ is the estimated value of $a$,
- $P(a[i])$ is the experimentally measured frequency of each value $a[i]$.

#### Combined Error:
To account for both theoretical and experimental sources of error, the best approach is to take the maximum of the two errors:

$$
\epsilon_a = \max(\epsilon_a^{QPE}, \epsilon_a^{P(a)}).
$$

This ensures that the computed error $\epsilon_a$ captures the worst-case scenario, whether it arises from limitations in the algorithm (ideal case) or noise in the quantum hardware (realistic case). By using this combined error metric, we can provide a robust estimate of the uncertainty in our amplitude estimation results.

In [ ]:
eps_a_pa = (qft_pe.result['Probability'] @ ((qft_pe.result['a'] - mean_a) ** 2))**0.5
eps_qpe = 2 * np.pi / 2 ** qft_pe.auxiliar_qbits_number
epsilon = max([eps_a_pa, eps_qpe])
print('QPE error: {} . Probability Density Error: {}'.format(eps_qpe, eps_a_pa))
print('Total Error: {}'.format(epsilon))

## 6. Amplitude Estimation with classical Quantum Phase Estimation

In order to simplify **Amplitude Estimation** calculations performed with the **CQPE** class (see Section 3), the **CQPEAE** class (which stands for **C**lassical **Q**uantum **P**hase **E**stimation **A**mplitude **Estimation**) was developed. This class addresses the initial complexities outlined in Section 5, enabling a more streamlined approach for using **CQPE** in *Amplitude Estimation* calculations. The `CQPEAE` class is implemented in the module **ae_classical_qpe** of the *Amplitude Estimation* package within the *QQuantLib* library (**QQuantLib/AE/ae_classical_qpe**).

#### Note:
The design of the `CQPEAE` class follows a similar structure to the `MLAE` class from the **QQuantLib/AE/maximum_likelihood_ae** module.

To create an instance of the `CQPEAE` class, the following inputs are mandatory:

1. `oracle`: A QLM `AbstractGate` or `QRoutine` that implements the oracle required for constructing the Grover operator.
2. `target`: The marked state in its binary representation, specified as a Python list.
3. `index`: A list of qubits that will be affected by the Grover operator.

These inputs ensure that the class is properly configured for executing amplitude estimation tasks efficiently and accurately.

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
#Here we created the mandatory oracle
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

#This will be the target state for grover and the list of qubits affected by Grover operator
target = [0, 0, 1]
index = range(oracle.arity)

Now that we have all the mandatory inputs, we can create the `CQPEAE` object. Following the conventions of the `MLAE` class, additional parameters can be provided through a Python dictionary. In this case, the following keys are available for configuration:

- `auxiliar_qbits_number`: The number of qubits used for phase estimation (default: 8).
- `shots`: The number of measurement shots to execute (default: 100).
- `qpu`: The QPU solver to be used.
- `mcz_qlm`: A boolean flag indicating whether to use the QLM multi-controlled Z gate (`True`, default) or the multiplexor implementation (`False`).

These parameters allow users to customize the behavior of the `CQPEAE` class according to their specific requirements and computational resources.

In [ ]:
from QQuantLib.AE.ae_classical_qpe import CQPEAE

In [ ]:
ae_pe_qft_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 4,
    'shots': 1000,
    'mcz_qlm': False    
}

ae_pe_qft = CQPEAE(
    oracle=oracle,
    target=target,
    index=index, 
    **ae_pe_qft_dict
)

When instantiated the `CQPEAE` class the *Grover* operator associated to the oracle operator is created. It can be access using attribute `_grover_oracle`.

In [ ]:
grover_circ = ae_pe_qft._grover_oracle
%qatdisplay grover_circ --depth 2 --svg

Now the `ae_pe_qft` object has all the mandatory inputs required to call the `CQPE` class and execute the algorithm. The `run` method of the `CQPEAE` class performs this step. Additionally, when the `run` method is executed, the following properties of the `CQPEAE` class are populated:

- `cqpe`: This property is an object from the **CQPE** class, initialized with the corresponding arguments created by the **CQPEAE** class.
- `final_results`: These are the final results obtained from the `pe_qft` method of the `cqpe` object (see Section 2.3).
- `theta`: The estimated value of $\theta$ obtained from the **CQPE** algorithm, computed as the mean of the probability distribution.
- `ae`: The amplitude estimation solution based on $a = \cos^2(\theta)$.
- `run_time`: The elapsed time for a complete execution of the **`run`** method.

Additionally, the computed errors for $a$ can be accessed using the following attributes:

- `epsilon_qpe`: Error due to the **QPE** algorithm.
- `epsilon_prob`: Statistical error derived from the performed measurements.
- `epsilon`: Final error, defined as the maximum of the two aforementioned errors ($\epsilon_qpe$ and $\epsilon_prob$).

In [ ]:
a_estimated = ae_pe_qft.run()

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_pe_withQFT.theta= ', ae_pe_qft.final_results)
print('ae_pe_withQFT.theta= ', ae_pe_qft.theta)
print('ae_pe_withQFT.a= ', ae_pe_qft.ae)
print("Classical result: ",probability[1])

print('QPE error: ', ae_pe_qft.epsilon_qpe)
print('Experimental error: ', ae_pe_qft.epsilon_prob)
print('Final error: ', ae_pe_qft.epsilon)

In [ ]:
print('Test OK: ',np.abs(ae_pe_qft.ae-probability[1])< 0.005)

In [ ]:
print('Elapsed time for run method: ',ae_pe_qft.run_time)

Of course the configuration of the algorithm can be changed by setting the attributes of the class an calling the *run* method again!!

In [ ]:
print(ae_pe_qft.auxiliar_qbits_number)

In [ ]:
ae_pe_qft.auxiliar_qbits_number = 10
print(ae_pe_qft.auxiliar_qbits_number)

In [ ]:
#run method again
a_estimated = ae_pe_qft.run()

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_pe_withQFT.theta= ', ae_pe_qft.theta)
print('ae_pe_withQFT.a= ', ae_pe_qft.ae)
print("Classical result: ",probability[1])

print('QPE error: ', ae_pe_qft.epsilon_qpe)
print('Experimental error: ', ae_pe_qft.epsilon_prob)
print('Final error: ', ae_pe_qft.epsilon)

In [ ]:
print('Elapsed time for run method: ',ae_pe_qft.run_time)

As explained before, the `cqpe` attribute from `CQPEAE` class is an object created from class `CQPE` so all the methods and attributes of this class can be accessed using the **cqpe** attribute (**NOT RECOMMENDED**).

We can access to the quantum circuit used by the **CQPE**

In [ ]:
#First we access to the attribute they stores the CQPE object
attribute_pewqtf = ae_pe_qft.cqpe
#Then we take the circuit attribute from the PhaseEstimationwQFT object (see above in the notebook)
c=attribute_pewqtf.circuit
%qatdisplay c --svg --depth 0

When the `run` method is executed following class attributes are populated:

* `circuit_statistics`: python dictionary with the complete statistical information of the circuit.
* `oracle_calls`: number of total oracle calls for a complete execution of the algorithm
* `max_oracle_depth`: maximum number of applications of the oracle for the complete execution of the algorithm.

In [ ]:
ae_pe_qft.circuit_statistics

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls for the CQPEAE was: {}".format(ae_pe_qft.oracle_calls))

In [ ]:
#Number of maximum oracle applications
ae_pe_qft.max_oracle_depth

In [ ]:
ae_pe_qft.quantum_times

In [ ]:
ae_pe_qft.quantum_time